## Section 1: Setup and Configuration

In [ ]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import os
from pathlib import Path
from collections import Counter, defaultdict
import json
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

print("Libraries imported successfully")

In [ ]:
# Cell 2: Configuration and Paths

# Research parameters - same as Notebook 1 for consistency
CRINK_COUNTRIES = {
    'CHINA': 'CHN',
    'RUSSIAN FEDERATION': 'RUS',
    'IRAN (ISLAMIC REPUBLIC OF)': 'IRN',
    "DEMOCRATIC PEOPLE'S REPUBLIC OF KOREA": 'PRK'
}

WESTERN_COUNTRIES = {
    'UNITED STATES': 'USA',
    'GERMANY': 'GER',
    'FRANCE': 'FRA',
    'UNITED KINGDOM': 'UK'
}

UNGA_START_YEAR = 1991
FIRST_COMMITTEE_START_YEAR = 2003
END_YEAR = 2024

PLOT_STYLE = 'bw'           # 'bw' or 'color'
FIGURE_SIZE = (14, 5)       # (width, height)
DPI = 300

# Setup paths
notebook_dir = Path.cwd()
repo_root = notebook_dir.parent
data_dir = repo_root / "data" / "processed"
results_dir = repo_root / "results"
mapping_dir = repo_root / "data" / "mappings"

data_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)
mapping_dir.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Results directory: {results_dir}")

## Section 2: Load Data

In [ ]:
# Cell 3: Load UN Voting Data

csv_files = list(data_dir.glob('*.csv'))
print(f"Available data files: {len(csv_files)}")

dataset_choice = 'UNGA_voting_records_filtered.csv'
csv_path = data_dir / dataset_choice

if not csv_path.exists():
    raise FileNotFoundError(
        f"Data file not found: {csv_path}\n"
        f"Please download from Harvard Dataverse and place in {data_dir}"
    )

print(f"Loading: {dataset_choice}")
df = pd.read_csv(csv_path, encoding='utf-8')

print(f"Data loaded: {df.shape}")

In [ ]:
# Cell 4: Data Preparation

if 'ms_name' in df.columns:
    df['ms_name'] = df['ms_name'].replace({'USSR': 'RUSSIAN FEDERATION'})

if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['year'] = df['date'].dt.year

df = df[(df['year'] >= UNGA_START_YEAR) & (df['year'] <= END_YEAR)].copy()

vote_pivot = df.pivot_table(
    index=['undl_id', 'date'],
    columns='ms_name',
    values='ms_vote',
    aggfunc='first'
).reset_index()

vote_pivot['year'] = pd.to_datetime(vote_pivot['date']).dt.year

print(f"Data prepared: {len(vote_pivot)} resolutions")

## Section 3: Advanced Alignment Analysis

In [ ]:
# Cell 5: Calculate CRINK Voting Patterns

crink_list = list(CRINK_COUNTRIES.keys())

vote_pivot['crink_votes'] = vote_pivot.apply(
    lambda row: [row[c] for c in crink_list if c in row.index and pd.notna(row[c])],
    axis=1
)

vote_pivot['crink_group_vote'] = vote_pivot['crink_votes'].apply(
    lambda votes: Counter(votes).most_common(1)[0][0] if votes else None
)

def get_agreement_count(votes):
    if len(votes) < 2:
        return None
    counts = Counter(votes)
    most_common_vote, count = counts.most_common(1)[0]
    return count if count >= 2 else None

vote_pivot['agreement_count'] = vote_pivot['crink_votes'].apply(get_agreement_count)

print(f"CRINK analysis complete")
print(f"  4-way unanimous: {(vote_pivot['agreement_count'] == 4).sum()}")
print(f"  3-way agreement: {(vote_pivot['agreement_count'] == 3).sum()}")
print(f"  2-way agreement: {(vote_pivot['agreement_count'] == 2).sum()}")

In [ ]:
# Cell 6: Western Democracies Analysis

western_list = list(WESTERN_COUNTRIES.keys())

vote_pivot['western_votes'] = vote_pivot.apply(
    lambda row: [row[c] for c in western_list if c in row.index and pd.notna(row[c])],
    axis=1
)

vote_pivot['western_group_vote'] = vote_pivot['western_votes'].apply(
    lambda votes: Counter(votes).most_common(1)[0][0] if votes else None
)

vote_pivot['western_agreement_count'] = vote_pivot['western_votes'].apply(get_agreement_count)

vote_pivot['crink_vs_western'] = (
    (vote_pivot['crink_group_vote'] == vote_pivot['western_group_vote']) & 
    vote_pivot['agreement_count'].notna() &
    vote_pivot['western_agreement_count'].notna()
)

if 'UNITED STATES' in vote_pivot.columns:
    vote_pivot['anti_us'] = (
        (vote_pivot['agreement_count'] == 4) & 
        (vote_pivot['crink_group_vote'] != vote_pivot['UNITED STATES']) &
        vote_pivot['UNITED STATES'].notna()
    )
    anti_us_count = vote_pivot['anti_us'].sum()
else:
    anti_us_count = 0

crink_western_agreement = vote_pivot['crink_vs_western'].sum()
crink_western_total = (vote_pivot['agreement_count'].notna() & vote_pivot['western_agreement_count'].notna()).sum()

print(f"Western democracies analysis complete")
if crink_western_total > 0:
    print(f"  CRINK-Western agreement: {crink_western_agreement}/{crink_western_total} ({100*crink_western_agreement/crink_western_total:.1f}%)")
print(f"  Anti-US votes (4-way CRINK vs US): {anti_us_count}")

## Section 4: Visualizations

In [ ]:
# Cell 7: Create Alignment Heatmap

all_countries = crink_list + western_list
alignment_matrix = pd.DataFrame(0.0, index=all_countries, columns=all_countries)

for i, country1 in enumerate(all_countries):
    for j, country2 in enumerate(all_countries):
        if country1 != country2 and country1 in vote_pivot.columns and country2 in vote_pivot.columns:
            both_voted = vote_pivot[
                (vote_pivot[country1].notna()) & (vote_pivot[country2].notna())
            ]
            if len(both_voted) > 0:
                alignment = 100 * (both_voted[country1] == both_voted[country2]).sum() / len(both_voted)
                alignment_matrix.iloc[i, j] = alignment

fig, ax = plt.subplots(figsize=(12, 10), dpi=100)

labels = [c.split()[0] for c in all_countries]
sns.heatmap(
    alignment_matrix,
    annot=True,
    fmt='.0f',
    cmap='RdYlGn' if PLOT_STYLE == 'color' else 'Greys',
    vmin=0,
    vmax=100,
    cbar_kws={'label': 'Alignment (%)'},
    ax=ax,
    xticklabels=labels,
    yticklabels=labels,
    linewidths=0.5
)
ax.set_title(
    'Dyadic Voting Alignment: CRINK vs Western Democracies (1991-2024)',
    fontsize=12, fontweight='bold', pad=20
)

plt.tight_layout()
plt.savefig(results_dir / 'figure_alignment_heatmap.png', dpi=DPI, bbox_inches='tight')
print(f"Saved: figure_alignment_heatmap.png")
plt.show()

## Section 5: Generate Tables

In [ ]:
# Cell 8: Generate Summary Tables

timestamp = pd.Timestamp.now().strftime('%Y%m%d')

# Table: Dyadic alignment matrix
table_path = results_dir / f'table_dyadic_alignment_{timestamp}.csv'
alignment_matrix.to_csv(table_path)
print(f"Saved: {table_path.name}")

# Table: Country pair statistics
pair_stats = []
for i, c1 in enumerate(all_countries):
    for c2 in all_countries[i+1:]:
        if c1 in vote_pivot.columns and c2 in vote_pivot.columns:
            both = vote_pivot[(vote_pivot[c1].notna()) & (vote_pivot[c2].notna())]
            if len(both) > 0:
                agreement = (both[c1] == both[c2]).sum()
                pair_stats.append({
                    'Country 1': c1.split()[0],
                    'Country 2': c2.split()[0],
                    'Total Votes': len(both),
                    'Agreement': agreement,
                    'Agreement %': 100 * agreement / len(both)
                })

pair_df = pd.DataFrame(pair_stats).sort_values('Agreement %', ascending=False)
pair_path = results_dir / f'table_pair_statistics_{timestamp}.csv'
pair_df.to_csv(pair_path, index=False)
print(f"Saved: {pair_path.name}")

print(f"\nAll tables saved to: {results_dir}")

## Section 6: Summary

In [ ]:
# Cell 9: Final Summary Report

print("\n" + "="*80)
print("CRINK ADVANCED ALIGNMENT ANALYSIS - SUMMARY REPORT")
print("="*80)

print(f"\nANALYSIS PERIOD: {UNGA_START_YEAR}-{END_YEAR}")
print(f"\nRESEARCH FOCUS:")
print(f"  Primary group: CRINK (China, Russia, Iran, North Korea)")
print(f"  Comparison group: Western democracies (US, Germany, France, UK)")

print(f"\nKEY FINDINGS:")
print(f"  Total resolutions analyzed: {len(vote_pivot)}")
print(f"  CRINK voting with 2+ agreement: {vote_pivot['agreement_count'].notna().sum()}")
print(f"  CRINK voting with 4-way (unanimous): {(vote_pivot['agreement_count'] == 4).sum()}")

if anti_us_count > 0:
    print(f"  Resolutions with 4-way CRINK against US: {anti_us_count}")

print(f"\nFIGURES GENERATED:")
print(f"  - Alignment heatmap (CRINK vs Western)")

print(f"\nTABLES GENERATED:")
print(f"  - Dyadic alignment matrix")
print(f"  - Country pair statistics")

print(f"\n" + "="*80)
print("Analysis complete. Ready for publication.")
print("="*80)